In [4]:
import os
from pprint import pprint

import mlflow
from mlflow import MlflowClient
from sklearn.ensemble import RandomForestRegressor

os.environ['MLFLOW_TRACKING_URI'] = './../mlruns'

# client = MlflowClient(tracking_uri="http://127.0.0.1:5000")


In [5]:
def create_mlflow_experiment(
    experiment_name: str, artifact_location: str, tags: dict[str, str]
) -> str:
    """
    Create a new mlflow experiment with the given name and artifact location.

    Parameters:
    ----------
    experiment_name: str
        The name of the experiment to create.
    artifact_location: str
        The artifact location of the experiment to create.
    tags: dict[str,Any]
        The tags of the experiment to create.

    Returns:
    -------
    experiment_id: str
        The id of the created experiment.
    """
    try:
        experiment_id = mlflow.create_experiment(
            name=experiment_name, artifact_location=artifact_location, tags=tags
        )
    except:
        print(f"Experiment {experiment_name} already exists.")
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    mlflow.set_experiment(experiment_name=experiment_name)

    return experiment_id


# create a new mlflow experiment
experiment_id = create_mlflow_experiment(
    experiment_name="testing_mlflow",
    artifact_location="testing_mlflow1_artifacts",
    tags={"env": "dev", "version": "1.0.0"},
)

print(experiment_id)


482949443120200499


In [6]:
def get_mlflow_experiment(
    experiment_id: str = None, experiment_name: str = None
) -> mlflow.entities.Experiment:
    """
    Retrieve the mlflow experiment with the given id or name.

    Parameters:
    ----------
    experiment_id: str
        The id of the experiment to retrieve.
    experiment_name: str
        The name of the experiment to retrieve.

    Returns:
    -------
    experiment: mlflow.entities.Experiment
        The mlflow experiment with the given id or name.
    """
    if experiment_id is not None:
        experiment = mlflow.get_experiment(experiment_id)
    elif experiment_name is not None:
        experiment = mlflow.get_experiment_by_name(experiment_name)
    else:
        raise ValueError("Either experiment_id or experiment_name must be provided.")

    return experiment


#retrieve the mlflow experiment
experiment = get_mlflow_experiment(experiment_id="434671109481285144")

print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
print("Creation timestamp: {}".format(experiment.creation_time))


Name: testing_mlflow10
Experiment_id: 434671109481285144
Artifact Location: /home/ubuntu/Bureau/OC/Projet5_oudelet_kevin/notebooks/testing_mlflow1_artifacts
Tags: {'env': 'dev', 'version': '1.0.0'}
Lifecycle_stage: active
Creation timestamp: 1705138449722


In [7]:
# On peut aussi delete depuis l'ui

mlflow.delete_experiment(experiment_id="434671109481285144")
